In [24]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 🔹 Función para buscar archivos
def find_file(filename):
    """Busca un archivo en la carpeta 'data' dentro del proyecto."""
    search_dir = Path.cwd()
    for parent in [search_dir] + list(search_dir.parents):
        possible_path = parent / "data" / filename
        if possible_path.exists():
            return possible_path.resolve()
    raise FileNotFoundError(f"❌ No se encontró el archivo {filename} en la estructura de directorios.")

# 🔹 Cargar dataset limpio
def load_clean_data(file_name):
    """Carga el dataset en formato Feather."""
    file_path = find_file(file_name)
    return pd.read_feather(file_path)

# 🔹 Feature Engineering
def feature_engineering(df, is_test=False):
    """Genera características adicionales para el modelo."""
    
    # Extraer día de la semana y hora del viaje
    df["day_of_week"] = df["start_time"].dt.dayofweek
    df["hour"] = df["start_time"].dt.hour

    # Marcar si es fin de semana
    df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

    # Crear categorías de duración de viaje
    df["duration_category"] = pd.cut(df["duration"],
                                     bins=[0, 5, 15, 30, 60, 120, 360],
                                     labels=["0-5 min", "5-15 min", "15-30 min", "30-60 min", "60-120 min", "120+ min"])

    # Conversión de `year_month` a numérico
    if "year_month" in df.columns:
        df["year_month"] = df["year_month"].str.replace("-", "").astype(int)

    # ❌ Eliminar columnas irrelevantes (incluyendo `plan_duration`)
    cols_to_drop = ["bike_id", "start_time", "end_time", "plan_duration"]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

    # Convertir variables categóricas a numéricas
    categorical_cols = ["trip_route_category", "start_station", "end_station", "duration_category"]

    if not is_test:
        categorical_cols.append("passholder_type")

    label_encoders = {}
    for col in categorical_cols:
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col].astype(str))
        label_encoders[col] = encoder

    return df, label_encoders

# 🔹 Procesar el dataset de entrenamiento (ahora sin `plan_duration`)
df_clean = load_clean_data("df_clean.feather")
df_features, label_encoders = feature_engineering(df_clean)

# Separar variables predictoras (X) y variable objetivo (y)
X = df_features.drop(columns=["passholder_type"])
y = df_features["passholder_type"]

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Guardar datasets procesados
data_dir = find_file("df_clean.feather").parent
train_data_path = data_dir / "train.feather"
test_data_path = data_dir / "test.feather"

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_feather(train_data_path)
test_data.to_feather(test_data_path)

print(f"✅ Feature engineering completado. Archivos guardados en:\n - {train_data_path}\n - {test_data_path}")


✅ Feature engineering completado. Archivos guardados en:
 - C:\Users\candy\Downloads\arkon_ds\data\train.feather
 - C:\Users\candy\Downloads\arkon_ds\data\test.feather


In [25]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 🔹 Cargar datasets preprocesados
try:
    train_path = find_file("train.feather")
    test_path = find_file("test.feather")
    print(f"✅ Train dataset encontrado en: {train_path}")
    print(f"✅ Test dataset encontrado en: {test_path}")

    df_train = pd.read_feather(train_path)
    df_test = pd.read_feather(test_path)

except FileNotFoundError as e:
    print(e)
    exit(1)

# 🔹 Separar variables predictoras (X) y variable objetivo (y)
X_train = df_train.drop(columns=["passholder_type"])
y_train = df_train["passholder_type"]

X_test = df_test.drop(columns=["passholder_type"])
y_test = df_test["passholder_type"]

# 🔹 Definir y entrenar el modelo (Random Forest optimizado)
model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# 🔹 Guardar el modelo entrenado
model_path = train_path.parent / "random_forest_model.pkl"
joblib.dump(model, model_path)
print(f"\n✅ Modelo guardado en: {model_path}")







✅ Train dataset encontrado en: C:\Users\candy\Downloads\arkon_ds\data\train.feather
✅ Test dataset encontrado en: C:\Users\candy\Downloads\arkon_ds\data\test.feather

✅ Modelo guardado en: C:\Users\candy\Downloads\arkon_ds\data\random_forest_model.pkl


In [29]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# 🔹 Función para buscar archivos en la estructura de directorios
def find_file(filename):
    """Busca un archivo en la carpeta 'data' dentro del proyecto."""
    search_dir = Path.cwd()
    for parent in [search_dir] + list(search_dir.parents):
        possible_path = parent / "data" / filename
        if possible_path.exists():
            return possible_path.resolve()
    raise FileNotFoundError(f"❌ No se encontró el archivo {filename} en la estructura de directorios.")

# 🔹 Cargar el modelo entrenado
try:
    model_path = find_file("random_forest_model.pkl")
    print(f"✅ Modelo encontrado en: {model_path}")
    model = joblib.load(model_path)
except FileNotFoundError as e:
    print(e)
    exit(1)

# 🔹 Cargar el dataset `df_clean_test.feather`
try:
    test_data_path = find_file("df_clean_test.feather")
    print(f"✅ Dataset de test encontrado en: {test_data_path}")
    df_test = pd.read_feather(test_data_path)
except FileNotFoundError as e:
    print(e)
    exit(1)

# 🔹 Feature Engineering en `df_clean_test.feather`
def feature_engineering(df):
    """Transforma el dataset para que coincida con el usado en entrenamiento."""
    
    # Extraer día de la semana y hora del viaje
    df["day_of_week"] = df["start_time"].dt.dayofweek
    df["hour"] = df["start_time"].dt.hour

    # Marcar si es fin de semana
    df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

    # Extraer año y mes en formato numérico
    df["year_month"] = df["start_time"].dt.strftime("%Y%m").astype(int)
    df["year"] = df["start_time"].dt.year.astype(int)

    # Categorizar duración del viaje
    df["duration_category"] = pd.cut(df["duration"],
                                     bins=[0, 5, 15, 30, 60, 120, 360],
                                     labels=[0, 1, 2, 3, 4, 5]).astype("int32")

    # 🔹 Asegurar que todas las columnas categóricas sean convertidas a numéricas
    categorical_cols = ["trip_route_category", "start_station", "end_station"]
    
    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].astype("category").cat.codes  # Convertir a numérico

    # Eliminar columnas irrelevantes
    df.drop(columns=["bike_id", "start_time", "end_time", "plan_duration"], inplace=True, errors="ignore")

    return df

# 🔹 Aplicar Feature Engineering al dataset de test
df_test = feature_engineering(df_test)

# 🔹 Verificar si `passholder_type` existe en `df_clean_test.feather`
if "passholder_type" in df_test.columns:
    y_test = df_test["passholder_type"]
    df_test.drop(columns=["passholder_type"], inplace=True)  # Lo eliminamos para que coincida con entrenamiento
    print("⚠️ `passholder_type` fue eliminado de `df_test` para evitar problemas en la predicción.")
else:
    y_test = None  # No hay etiquetas reales, solo podemos hacer predicciones sin evaluación

# 🔹 Asegurar que las columnas coincidan con las del modelo entrenado
expected_features = list(model.feature_names_in_)  # Mantener el orden de las columnas
df_test = df_test[expected_features]  # Reordenar `df_test` según entrenamiento

# 🔹 Realizar predicciones con el modelo entrenado
y_pred = model.predict(df_test)

# 🔹 Evaluar el desempeño del modelo en `df_clean_test.feather`
if y_test is not None:
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)

    print(f"\n✅ **Evaluación del Modelo en Datos No Vistos**")
    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1 Score: {f1:.4f}")

    print("\n🔹 Reporte de Clasificación:")
    print(classification_report(y_test, y_pred))

    # 🔹 Matriz de Confusión
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\n🔹 Matriz de Confusión:")
    print(conf_matrix)
else:
    print("\n⚠️ `df_clean_test.feather` no tiene `passholder_type`, por lo que no se pueden calcular métricas de evaluación.")
    print("✅ Predicciones generadas con éxito.")



✅ Modelo encontrado en: C:\Users\candy\Downloads\arkon_ds\data\random_forest_model.pkl
✅ Dataset de test encontrado en: C:\Users\candy\Downloads\arkon_ds\data\df_clean_test.feather

⚠️ `df_clean_test.feather` no tiene `passholder_type`, por lo que no se pueden calcular métricas de evaluación.
✅ Predicciones generadas con éxito.


In [30]:
# 🔹 Guardar las predicciones en un archivo CSV
df_test["predicted_passholder_type"] = y_pred
output_path = test_data_path.parent / "predictions.csv"
df_test.to_csv(output_path, index=False)

print(f"✅ Predicciones guardadas en: {output_path}")

✅ Predicciones guardadas en: C:\Users\candy\Downloads\arkon_ds\data\predictions.csv
